In [1]:
import pyspark
from pyspark.sql import SparkSession

In [6]:
spark = SparkSession.builder \
    .master('local[*]') \
    .appName('test') \
    .getOrCreate()

24/03/03 21:17:05 WARN Utils: Your hostname, DESKTOP-87NE6FD resolves to a loopback address: 127.0.1.1; using 172.28.203.119 instead (on interface eth0)
24/03/03 21:17:05 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


24/03/03 21:17:08 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


load green taxi data, made a report with SQL and save report to paqrquet

In [67]:
df_green = spark.read.parquet('data/pq/green/*/*')

In [9]:
df_green.columns

['VendorID',
 'lpep_pickup_datetime',
 'lpep_dropoff_datetime',
 'store_and_fwd_flag',
 'RatecodeID',
 'PULocationID',
 'DOLocationID',
 'passenger_count',
 'trip_distance',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'ehail_fee',
 'improvement_surcharge',
 'total_amount',
 'payment_type',
 'trip_type',
 'congestion_surcharge']

In [68]:
df_green.registerTempTable('green')

In [69]:
df_green_revenue = spark.sql("""
SELECT
    date_trunc('hour', lpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_of_records
FROM
    green
WHERE
    lpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [70]:
df_green_revenue.show()

+-------------------+----+------------------+-----------------+
|               hour|zone|            amount|number_of_records|
+-------------------+----+------------------+-----------------+
|2020-01-02 14:00:00|  26|            327.48|               11|
|2020-01-06 15:00:00| 241|             184.7|                3|
|2020-01-22 13:00:00|  74|            666.24|               44|
|2020-01-14 23:00:00| 223|             48.31|                4|
|2020-01-09 23:00:00| 260|202.32999999999998|               16|
|2020-01-30 09:00:00|  75|1010.3499999999992|               67|
|2020-01-11 21:00:00|  41| 700.5999999999996|               50|
|2020-01-07 18:00:00|  45|            112.39|                4|
|2020-01-02 09:00:00|  42| 555.7500000000002|               33|
|2020-01-04 23:00:00|  25| 376.3999999999999|               26|
|2020-01-25 11:00:00| 259|            161.21|                3|
|2020-01-16 20:00:00|  52|             108.6|                7|
|2020-01-16 06:00:00|  74| 600.800000000

save the result to parquet

In [31]:
df_green_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/green', mode='overwrite')

24/03/03 21:52:32 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


load yellow taxi data, made a report with SQL and save report to paqrquet

In [71]:
df_yellow = spark.read.parquet('data/pq/yellow/*/*')

In [72]:
df_yellow.columns

['VendorID',
 'tpep_pickup_datetime',
 'tpep_dropoff_datetime',
 'passenger_count',
 'trip_distance',
 'RatecodeID',
 'store_and_fwd_flag',
 'PULocationID',
 'DOLocationID',
 'payment_type',
 'fare_amount',
 'extra',
 'mta_tax',
 'tip_amount',
 'tolls_amount',
 'improvement_surcharge',
 'total_amount',
 'congestion_surcharge']

In [73]:
df_yellow.registerTempTable('yellow')

In [74]:
df_yellow_revenue = spark.sql("""
SELECT
    date_trunc('hour', tpep_pickup_datetime) AS hour, 
    PULocationID AS zone,
    SUM(total_amount) AS amount,
    COUNT(1) AS number_of_records
FROM
    yellow
WHERE
    tpep_pickup_datetime >= '2020-01-01 00:00:00'
GROUP BY
    1, 2
""")

In [75]:
df_yellow_revenue.show()

+-------------------+----+------------------+-----------------+
|               hour|zone|            amount|number_of_records|
+-------------------+----+------------------+-----------------+
|2020-01-16 18:00:00| 232|261.71000000000004|               12|
|2020-01-06 05:00:00|  48|2105.5099999999984|              136|
|2020-01-03 15:00:00| 163| 5261.330000000002|              348|
|2020-01-06 11:00:00|  43|2731.4199999999983|              185|
|2020-01-17 07:00:00|  24|           1058.45|               62|
|2020-01-12 11:00:00| 163| 4039.869999999997|              244|
|2020-01-20 20:00:00| 144|1758.8999999999996|              107|
|2020-01-27 18:00:00| 162|11701.700000000033|              698|
|2020-01-26 03:00:00| 263|           1156.81|               69|
|2020-01-20 18:00:00| 113|2801.8799999999987|              197|
|2020-01-24 15:00:00| 186|7813.4200000000155|              446|
|2020-01-26 16:00:00| 236| 7387.040000000016|              509|
|2020-01-09 18:00:00| 263| 4200.28999999

In [46]:
df_yellow_revenue \
    .repartition(20) \
    .write.parquet('data/report/revenue/yellow', mode='overwrite')

24/03/03 21:56:25 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


In [79]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_of_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_of_records', 'yellow_number_records')

In [80]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [81]:
df_join

DataFrame[hour: timestamp, zone: int, green_amount: double, green_number_records: bigint, yellow_amount: double, yellow_number_records: bigint]

In [82]:
df_join.show()

+-------------------+----+------------------+--------------------+------------------+---------------------+
|               hour|zone|      green_amount|green_number_records|     yellow_amount|yellow_number_records|
+-------------------+----+------------------+--------------------+------------------+---------------------+
|2020-01-01 00:00:00|  29|              61.3|                   1|              null|                 null|
|2020-01-01 00:00:00|  33|317.27000000000004|                  11|            255.56|                    8|
|2020-01-01 00:00:00|  62|             15.95|                   1|             61.43|                    1|
|2020-01-01 00:00:00|  71|              23.8|                   1|              null|                 null|
|2020-01-01 00:00:00|  81|54.870000000000005|                   2|             30.32|                    1|
|2020-01-01 00:00:00| 125|              null|                null|           1342.07|                   68|
|2020-01-01 00:00:00| 126|  

In [83]:
df_join.write.parquet('data/report/revenue/total')

24/03/03 22:40:27 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers


now instead of compute step by step we can just loading our data which we've compute earlier 

In [84]:
df_green_revenue = spark.read.parquet('data/report/revenue/green')
df_yellow_revenue = spark.read.parquet('data/report/revenue/yellow')

In [85]:
df_green_revenue_tmp = df_green_revenue \
    .withColumnRenamed('amount', 'green_amount') \
    .withColumnRenamed('number_of_records', 'green_number_records')

df_yellow_revenue_tmp = df_yellow_revenue \
    .withColumnRenamed('amount', 'yellow_amount') \
    .withColumnRenamed('number_of_records', 'yellow_number_records')

In [86]:
df_join = df_green_revenue_tmp.join(df_yellow_revenue_tmp, on=['hour', 'zone'], how='outer')

In [88]:
df_join.write.parquet('data/report/revenue/total', mode='overwrite')

24/03/03 22:54:56 WARN MemoryManager: Total allocation exceeds 95.00% (1,020,054,720 bytes) of heap memory
Scaling row group sizes to 95.00% for 8 writers
